In [1]:
import pandas as pd
from dotenv import dotenv_values, load_dotenv
import requests
import json

In [2]:
pd.__version__

'2.1.1'

## German Census Database (Genesis) – Create Table of Contents via API
  
*creating the complete Excel file may take up to 90 minutes*

for full api documentation (in German), see  
https://ergebnisse2011.zensus2022.de/datenbank/misc/ZENSUS-Webservices_Einfuehrung.pdf

code inspiration thanks to  
https://github.com/sjockers/genesis-api-example

In [3]:
# set preferred language (de|en)
langPref = "en"
# expand text output
pd.set_option('display.max_colwidth', None)

In [4]:
# Load credentials from .env file
load_dotenv()  
USERNAME, PASSWORD = dotenv_values().values()

# Set base path for API calls
BASE_URL = 'https://ergebnisse2011.zensus2022.de/api/rest/2020/'

### Check login credentials

In [5]:
hello = requests.get(BASE_URL + 'helloworld/logincheck', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': langPref
})
hello.json()["Status"]

'You have been logged in and out successfully!'

### Catalogue Service

In [6]:
variables = requests.get(BASE_URL + 'catalogue/variables', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': langPref,
    'pagelength': 200
})
variableCodes = variables.json()["List"]

In [7]:
len(variableCodes)

134

In [8]:
variableCodes[-3:]

[{'Code': 'WHGFL3',
  'Content': 'Floor area of the dwelling (20m² intervals)',
  'Type': 'Subject',
  'Values': '10',
  'Information': 'true'},
 {'Code': 'WHGNZ1',
  'Content': 'Type of use of the dwelling',
  'Type': 'Subject',
  'Values': '5',
  'Information': 'true'},
 {'Code': 'WHGRM2',
  'Content': 'Rooms',
  'Type': 'Subject',
  'Values': '7',
  'Information': 'true'}]

In [287]:
def metaVal(var):

    metadata = requests.get(BASE_URL + 'catalogue/values2variable', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'pagelength': 15000,  # there are 11.340 municipality names as of 2011
        'name': var
    })
    
    return(metadata.json()["List"])

In [288]:
metadataValues = pd.DataFrame()

for obj in variableCodes:

    myList = metaVal(obj["Code"])

    try:
        len(myList)
    except:
        myList = [{"Code":"","Content":""}]
    
    for item in myList:
        
        metadataValues = pd.concat([metadataValues, pd.DataFrame({ "Variable": obj["Code"], \
                                                                  "VarLabel": obj["Content"], \
                                                                  "Code": item["Code"], \
                                                                  "Content": item["Content"]}, \
                                                                     index=[0])])

In [289]:
metadataValues

,Variable,VarLabel,Code,Content
0,ALTER1,Age (yearly stages),ALTERU01,Under 1 year
0,ALTER1,Age (yearly stages),ALTER001,1 year
0,ALTER1,Age (yearly stages),ALTER002,2 years
0,ALTER1,Age (yearly stages),ALTER003,3 years
0,ALTER1,Age (yearly stages),ALTER004,4 years
...,...,...,...,...
0,WHGRM2,Rooms,RAUM03,3 rooms
0,WHGRM2,Rooms,RAUM04,4 rooms
0,WHGRM2,Rooms,RAUM05,5 rooms
0,WHGRM2,Rooms,RAUM06,6 rooms


In [112]:
tables = requests.get(BASE_URL + 'catalogue/tables', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': langPref,
    'pagelength': 1500
})
tableCodes = tables.json()["List"]

In [113]:
len(tableCodes)

754

In [114]:
tableCodes[-3:]

[{'Code': '6000F-2004',
  'Content': 'Families: Size of family nucleus/Type of family nucleus\n(by family) - Equipment in dwelling/Floor area of the\ndwelling (20m² intervals)/Rooms',
  'Time': '2011-05-09 to 2011-05-09'},
 {'Code': '6000F-3001',
  'Content': 'Families: Size of private household - Senior citizen status\n- Type of private household (by family)',
  'Time': '2011-05-09 to 2011-05-09'},
 {'Code': '6000F-3002',
  'Content': 'Families: Size of family nucleus - Type of family nucleus\n(by family) - Equipment in dwelling/Floor area of the\ndwelling (20m² intervals)/Rooms',
  'Time': '2011-05-09 to 2011-05-09'}]

### Metadata Service (Variables)

In [25]:
varCodes=[]

In [26]:
for item in variableCodes:
    varCodes.append(item.get("Code"))

In [27]:
varCodes[0:5]

['ALTER1', 'ALTGR1', 'ALTGR2', 'ALTGR3', 'ALTGR4']

In [49]:
def metaVar(var):

    metadata = requests.get(BASE_URL + 'metadata/variable', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'name': var
    })
    
    return(metadata.json()["Object"])

In [284]:
metadataVariable = pd.DataFrame()

for code in varCodes:

    myObject = metaVar(code)
    
    try:
        myInformation = myObject["Information"].replace("wiki","").replace("\n"," ")
    except:
        myInformation = ""

    metadataVariable = pd.concat([metadataVariable, pd.DataFrame({"Code": myObject["Code"], \
                                                          "Content": myObject["Content"], \
                                                          "Values" : myObject["Values"], \
                                                          "Information": myInformation, \
                                                          "Updated": myObject["Updated"]},\
                                                          index=[0])])

In [285]:
metadataVariable.head(3)

,Code,Content,Values,Information,Updated
0,ALTER1,Age (yearly stages),101,The age of the person refers to the completed year of life on 9 May 2011. Broken down into individual years of age.,2022-05-09 15:37:41h
0,ALTGR1,Age (five years age groups),19,The age of the person refers to the completed year of life on 9 May 2011. This variable indicates a person's age group for reference date 9 May 2011. The ages are cited in five years age groups.,2020-10-28 10:42:15h
0,ALTGR2,Age (ten years age groups),9,The age of the person refers to the completed year of life on 9 May 2011. This variable indicates a person's age group for reference date 9 May 2011. The ages are cited in ten years age groups.,2020-10-28 10:42:19h


In [291]:
def metaTab2Var(var):

    metadata = requests.get(BASE_URL + 'catalogue/tables2variable', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'pagelength': 1000,    # variables like GEODL1 occur in almost all tables
        'name': var
    })

    myList = metadata.json()["List"]
    
    try:
        len(myList)
    except:
        myList = [{"Code": "-"}]

    myTables = []
    for item in myList:
        myTables.append(item.get("Code"))

    return(myTables)

In [300]:
# get a list of tables where each variable is available
# this might take an hour
metadataVariable["TableList"] = metadataVariable["Code"].apply(lambda x: metaTab2Var(x))

In [302]:
metadataVariable["TableCount"] = metadataVariable["TableList"].apply(lambda x: len(x))

In [303]:
# convert list to string for prettier handling in Excel
def list2str(myListObj):
    myL = pd.Series(myListObj)
    return(myL.to_string(index=False).replace("\n",", "))

In [304]:
metadataVariable["TableListStr"] = metadataVariable["TableList"].apply(lambda x: list2str(x))

In [305]:
metadataVariable.head(3)

,Code,Content,Values,Information,Updated,TableList,TableCount,TableListStr
0,ALTER1,Age (yearly stages),101,The age of the person refers to the completed year of life on 9 May 2011. Broken down into individual years of age.,2022-05-09 15:37:41h,"[1000A-1006, 1000A-2007, 1000A-2008, 1000A-2012, 1000A-3010, 2000S-2015, 2000S-2082, 2000S-2086, 2000S-3028, 2000S-3169, 2000S-3173]",11,"1000A-1006, 1000A-2007, 1000A-2008, 1000A-2012, 1000A-3010, 2000S-2015, 2000S-2082, 2000S-2086, 2000S-3028, 2000S-3169, 2000S-3173"
0,ALTGR1,Age (five years age groups),19,The age of the person refers to the completed year of life on 9 May 2011. This variable indicates a person's age group for reference date 9 May 2011. The ages are cited in five years age groups.,2020-10-28 10:42:15h,"[1000A-1004, 1000A-2005, 1000A-2006, 1000A-2012, 1000A-2024, 1000A-2027, 1000A-2028, 1000A-2044, 1000A-3005, 1000A-3011, 1000A-3018, 1000A-3020, 1000A-3026, 1000A-3027, 1000A-3028, 1000A-3040, 1000A-3041, 1000A-3042, 1000A-4005, 1000A-4006, 1000A-4007, 1000A-4012, 1000A-4013, 1000A-4014, 1000A-4015, 1000A-5001, 2000S-2001, 2000S-2006, 2000S-2009, 2000S-2021, 2000S-2023, 2000S-2027, 2000S-2037, 2000S-2077, 2000S-3001, 2000S-3008, 2000S-3009, 2000S-3010, 2000S-3015, 2000S-3018, 2000S-3021, 2000S-3023, 2000S-3041, 2000S-3042, 2000S-3045, 2000S-3046, 2000S-3047, 2000S-3056, 2000S-3057, 2000S-3058, 2000S-3060, 2000S-3065, 2000S-3072, 2000S-3073, 2000S-3162, 2000S-4001, 2000S-4002, 2000S-4010, 2000S-4012, 2000S-4013, 2000S-4014, 2000S-4015, 2000S-4026, 2000S-4027, 2000S-4028, 2000S-4030, 2000S-4031, 2000S-4037, 2000S-4039, 2000S-4042, 2000S-4047, 2000S-4048, 2000S-5001, 2000S-5003, 2000S-5004, 2000S-5006, 2000S-5012]",77,"1000A-1004, 1000A-2005, 1000A-2006, 1000A-2012, 1000A-2024, 1000A-2027, 1000A-2028, 1000A-2044, 1000A-3005, 1000A-3011, 1000A-3018, 1000A-3020, 1000A-3026, 1000A-3027, 1000A-3028, 1000A-3040, 1000A-3041, 1000A-3042, 1000A-4005, 1000A-4006, 1000A-4007, 1000A-4012, 1000A-4013, 1000A-4014, 1000A-4015, 1000A-5001, 2000S-2001, 2000S-2006, 2000S-2009, 2000S-2021, 2000S-2023, 2000S-2027, 2000S-2037, 2000S-2077, 2000S-3001, 2000S-3008, 2000S-3009, 2000S-3010, 2000S-3015, 2000S-3018, 2000S-3021, 2000S-3023, 2000S-3041, 2000S-3042, 2000S-3045, 2000S-3046, 2000S-3047, 2000S-3056, 2000S-3057, 2000S-3058, 2000S-3060, 2000S-3065, 2000S-3072, 2000S-3073, 2000S-3162, 2000S-4001, 2000S-4002, 2000S-4010, 2000S-4012, 2000S-4013, 2000S-4014, 2000S-4015, 2000S-4026, 2000S-4027, 2000S-4028, 2000S-4030, 2000S-4031, 2000S-4037, 2000S-4039, 2000S-4042, 2000S-4047, 2000S-4048, 2000S-5001, 2000S-5003, 2000S-5004, 2000S-5006, 2000S-5012"
0,ALTGR2,Age (ten years age groups),9,The age of the person refers to the completed year of life on 9 May 2011. This variable indicates a person's age group for reference date 9 May 2011. The ages are cited in ten years age groups.,2020-10-28 10:42:19h,"[1000A-1001, 1000A-2001, 1000A-2002, 1000A-2012, 1000A-2022, 1000A-2023, 1000A-2024, 1000A-2043, 1000A-3001, 1000A-3004, 1000A-3005, 1000A-3006, 1000A-3007, 1000A-3009, 1000A-3015, 1000A-3021, 1000A-3022, 1000A-3023, 1000A-3037, 1000A-3038, 1000A-4001, 1000A-4002, 1000A-4003, 1000A-4011, 2000S-2002, 2000S-2005, 2000S-2011, 2000S-2014, 2000S-2020, 2000S-2023, 2000S-2026, 2000S-2034, 2000S-2042, 2000S-2046, 2000S-2049, 2000S-2053, 2000S-2056, 2000S-2059, 2000S-2068, 2000S-2071, 2000S-2075, 2000S-2085, 2000S-2090, 2000S-3002, 2000S-3007, 2000S-3010, 2000S-3019, 2000S-3024, 2000S-3025, 2000S-3026, 2000S-3039, 2000S-3042, 2000S-3046, 2000S-3048, 2000S-3049, 2000S-3053, 2000S-3054, 2000S-3070, 2000S-3071, 2000S-3074, 2000S-3081, 2000S-3082, 2000S-3090, 2000S-3091, 2000S-3097, 2000S-3100, 2000S-3101, 2000S-3104, 2000S-3106, 2000S-3109, 2000S-3111, 2000S-3114, 2000S-3115, 2000S-3131, 2000S-3132, 2000S-3135, 2000S-3145, 2000S-3147, 2000S-3150, 2000S-3153, 2000S-3155, 2000S-3168, 2000S-3176, 2000S-3177, 2000S-3178, 2000S-4003, 2000S-4018, 2000S-4026, 2000S-4032, 2000S-4033, 2000S-4051, 2000S-4052, 2000S-4056, 20

### Metadata Service (Tables)

In [115]:
tabCodes=[]

In [116]:
for item in tableCodes:
    tabCodes.append(item.get("Code"))

In [117]:
tabCodes[-5:]

['6000F-2002', '6000F-2003', '6000F-2004', '6000F-3001', '6000F-3002']

In [118]:
def metaTab(tab):

    metadata = requests.get(BASE_URL + 'metadata/table', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'name': tab
    })
    
    return(metadata.json()["Object"])

In [120]:
metadataTable = pd.DataFrame()

# get metadata for all tables (754 for the 2011 census) - this might take more than an hour

for code in tabCodes:

    myObject = metaTab(code)
    
    try:
        myCol2= myObject["Structure"]["Columns"][1]["Code"]
    except:
        myCol2= ""
    try:
        myCol3= myObject["Structure"]["Columns"][2]["Code"]
    except:
        myCol3= ""
    
    try:
        myRow2= myObject["Structure"]["Rows"][0]["Structure"][0]["Code"]
    except:
        myRow2= ""     
    try:
        myRow3= myObject["Structure"]["Rows"][0]["Structure"][0]["Structure"][0]["Code"]
    except:
        myRow3= ""
    try:
        myRow4= myObject["Structure"]["Rows"][0]["Structure"][0]["Structure"][0]["Structure"][0]["Code"]
    except:
        myRow4= ""
    try:
        myRow5= myObject["Structure"]["Rows"][0]["Structure"][0]["Structure"][0]["Structure"][0]["Structure"][0]["Code"]
    except:
        myRow5= ""
        
    metadataTable = pd.concat([metadataTable, pd.DataFrame({"Code": myObject["Code"], \
                                                          "Content": myObject["Content"], \
                                                          "Col1": myObject["Structure"]["Columns"][0]["Code"], \
                                                          "Col2": myCol2, \
                                                          "Col3": myCol3, \
                                                          "Row1": myObject["Structure"]["Rows"][0]["Code"], \
                                                          "Row2": myRow2, \
                                                          "Row3": myRow3, \
                                                          "Row4": myRow4, \
                                                          "Row5": myRow5, \
                                                          "Updated": myObject["Updated"]},\
                                                          index=[0])])

In [232]:
metadataTable.head(3)

,Code,Content,Col1,Col2,Col3,Row1,Row2,Row3,Row4,Row5,Updated
0,1000A-0001,Persons: Official population and area (municipalities),PRS018,FLC001,PRS017,GEOGM1,,,,,2022-01-20 08:17:38h
0,1000A-1001,Persons: Age (ten years age groups),GEODL1,GEOBL1,,ALTGR2,,,,,2022-03-21 13:56:14h
0,1000A-1002,Persons: Age (eleven classes of years),GEODL1,GEOBL1,,ALTKL2,,,,,2022-03-21 13:56:14h


In [236]:
# cross reference which variables can be selected in a given table
# will be calculated from previously retrieved data, no api requests

metadataVar2Tab = pd.DataFrame()

for tcode in tabCodes:

    myVarList = []

    for vcode in varCodes:

        myTableList = metadataVariable[metadataVariable["Code"]== vcode]["TableList"].to_list()[0]

        if tcode in myTableList:
    
            myVarList.append(vcode)

    metadataVar2Tab = pd.concat([metadataVar2Tab, pd.DataFrame({"Code2": tcode, \
                                                          "VarCount": len(myVarList), \
                                                          "VarList": ', '.join(myVarList) },\
                                                          index=[0])])

In [237]:
metadataVar2Tab

,Code2,VarCount,VarList
0,1000A-0001,5,"FLC001, GEOGM1, PRS017, PRS018, STAG"
0,1000A-1001,10,"ALTGR2, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, PRS001, PRS018, STAG"
0,1000A-1002,10,"ALTKL2, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, PRS001, PRS018, STAG"
0,1000A-1003,10,"ALTKL1, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, PRS001, PRS018, STAG"
0,1000A-1004,10,"ALTGR1, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, PRS001, PRS018, STAG"
...,...,...,...
0,6000F-2002,11,"FAM001, GEOBL1, GEODL1, GEOGM2, GEOLK1, GEORB1, GEOVB3, HSHGR2, HSHSE1, HSHTP1, STAG"
0,6000F-2003,10,"FAM001, GEOBL1, GEODL1, GEOGM2, GEOLK1, GEORB1, GEOVB3, HSHSE1, HSHTP1, STAG"
0,6000F-2004,13,"FAM002, FAMGR2, FAMTP1, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, STAG, WHGAS1, WHGFL3, WHGRM2"
0,6000F-3001,11,"FAM001, GEOBL1, GEODL1, GEOGM2, GEOLK1, GEORB1, GEOVB3, HSHGR2, HSHSE1, HSHTP1, STAG"


In [238]:
metadataTable2 = metadataTable.set_index("Code").join(metadataVar2Tab.set_index("Code2"))

In [268]:
metadataTable2.head(3)

,Content,Col1,Col2,Col3,Row1,Row2,Row3,Row4,Row5,Updated,VarCount,VarList
Code,,,,,,,,,,,,
1000A-0001,Persons: Official population and area (municipalities),PRS018,FLC001,PRS017,GEOGM1,,,,,2022-01-20 08:17:38h,5,"FLC001, GEOGM1, PRS017, PRS018, STAG"
1000A-1001,Persons: Age (ten years age groups),GEODL1,GEOBL1,,ALTGR2,,,,,2022-03-21 13:56:14h,10,"ALTGR2, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, PRS001, PRS018, STAG"
1000A-1002,Persons: Age (eleven classes of years),GEODL1,GEOBL1,,ALTKL2,,,,,2022-03-21 13:56:14h,10,"ALTKL2, GEOBL1, GEODL1, GEOGM1, GEOLK1, GEORB1, GEOVB1, PRS001, PRS018, STAG"


## Export to a single styled Excel file

In [99]:
from openpyxl.styles import Alignment
from openpyxl.styles import DEFAULT_FONT
from openpyxl.styles import Font

In [100]:
_font = Font(name="Arial", sz=8)
defs = {k: setattr(DEFAULT_FONT, k, v) for k, v in _font.__dict__.items()}

In [306]:
with pd.ExcelWriter("census2011_toc_"+langPref+".xlsx") as writer:  

    metadataVariable[["Code", "Content", "Values", "Information", "Updated", "TableCount", "TableListStr"]]\
                    .to_excel(writer, sheet_name='variables', index=False)
    metadataValues.to_excel(writer, sheet_name='values', index=False)
    metadataTable2.to_excel(writer, sheet_name='tables', index=True)

    # some styling, could be skipped
    
    writer.book["variables"].column_dimensions["A"].width = 15
    writer.book["variables"].column_dimensions["B"].width = 25
    writer.book["variables"].column_dimensions["D"].width = 60
    writer.book["variables"].column_dimensions["E"].width = 20
    writer.book["variables"].column_dimensions["F"].width = 20
    writer.book["variables"].column_dimensions["G"].width = 150

    for idx in enumerate(metadataVariable["Code"]):
        
        writer.book["variables"].row_dimensions[idx[0]+2].height = 50
        writer.book["variables"]["B"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")
        writer.book["variables"]["D"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")
        writer.book["variables"]["G"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")
        writer.book["variables"]["C"+str(idx[0]+2)].alignment = Alignment(wrap_text=False, horizontal="center")
        writer.book["variables"]["E"+str(idx[0]+2)].alignment = Alignment(wrap_text=False, horizontal="center")
        writer.book["variables"]["F"+str(idx[0]+2)].alignment = Alignment(wrap_text=False, horizontal="center")

    writer.book["values"].column_dimensions["A"].width = 15
    writer.book["values"].column_dimensions["B"].width = 50
    writer.book["values"].column_dimensions["C"].width = 25
    writer.book["values"].column_dimensions["D"].width = 50

    writer.book["tables"].column_dimensions["A"].width = 15
    writer.book["tables"].column_dimensions["B"].width = 50
    writer.book["tables"].column_dimensions["K"].width = 20
    writer.book["tables"].column_dimensions["L"].width = 15
    writer.book["tables"].column_dimensions["M"].width = 90

    for idx in enumerate(metadataTable["Code"]):
        
        writer.book["tables"].row_dimensions[idx[0]+2].height = 35
        writer.book["tables"]["A"+str(idx[0]+2)].alignment = Alignment(wrap_text=False, vertical="bottom")
        writer.book["tables"]["B"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")
        writer.book["tables"]["K"+str(idx[0]+2)].alignment = Alignment(wrap_text=False, horizontal="center")
        writer.book["tables"]["L"+str(idx[0]+2)].alignment = Alignment(wrap_text=False, horizontal="center")
        writer.book["tables"]["M"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")